# DLC/ANIPOSE PIPELINE

Automatically converts fly videos to estimated 3D coordinates using DeepLabCut and Anipose

In [2]:
# Next two lines are for dev purposes only 
%load_ext autoreload 
%autoreload 2

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("Logging works :)")

from pipeline.pipeline_step_1 import run_preprocessing, analyze_new

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


Loading DLC 2.3.5...


### STEP 0: Configuration

- Set all necessary variables (videos path and root path) in `pipeline/config.yml`!

#### Override
If you would like to set these values in the noetbook, you can instead 
- Add filepath to DLC network config files (e.g `dlc_networks.yml`)
- Add filepath to folder with experiment videos

In [33]:
"""NOTE: this is just for overriding the config parameters within notebook"""

# Set the filepath to the genotype folder containing videos
# videos = Path(r"\\mpfi.org\public\sb-lab\DLC_pipeline_Dummy\0_QualityCtrl\Raw_vid_test_new_Code_2\40_BDN1_CsChrim_decap")

# root directory of project (usually overall experiment folder or similar)
# root = Path(r'\\mpfi.org\public\sb-lab\DLC_pipeline_Dummy\0_QualityCtrl') # Root directory of experiment/run, typically genotype or some other overarching directory. 
# Videos must be in dir or a subdir of root 

### STEP 1: Running DeepLabCut

This will run on a directory with fly video files and generate DLC pose estimations, outputting to the same directory.

##### Processing done:
- DeepLabCut `analyze_videos`
- DeepLabCut `filterpredictions`

In [3]:
analyze_new()
print("Finished DLC analysis...")

INFO:root:Searching through C:\Users\ryabinkyj\Documents\testanalyze\RawData\BIN-1
INFO:root:Found 3 Ball folders
INFO:root:Found 8 MP4 files
INFO:root:Analyzing movie: A-04192023182728-0000.mp4


INFO:root:Analyzing movie: B-04192023182729-0000.mp4
INFO:root:Analyzing movie: D-04192023182727-0000.mp4
INFO:root:Analyzing movie: E-04192023182730-0000.mp4
INFO:root:Analyzing movie: G-04192023182752-0000.mp4
INFO:root:Skipping video file: model path empty for Camera G
INFO:root:Analyzing movie: H-04192023182753-0000.mp4
INFO:root:Analyzing movie: skipC-04192023182749-0000skipsk.mp4
INFO:root:Analyzing movie: skipF-04192023182750-0000.mp4
INFO:root:Found 0 MP4 files
INFO:root:Found 0 MP4 files




Finished DLC analysis...


### STEP 2: Preprocessing for anipose

##### Processing done:
- Fix points
- Remove columns
- Rename (GenotypeFly#-CamName)
- Convert to .h5
- Generate anipose file structure

In [6]:
run_preprocessing()

INFO:pipeline.pipeline_step_1:Generating anipose files...


Finished preprocessing...
